In [ ]:
#!/usr/bin/env python3
"""
Minimal script to ping a Neo4J database and update "lastPing" property of a node type "Ping" (id=1)
"""
import os
from datetime import datetime
from neo4j import GraphDatabase, basic_auth

URI = os.environ["NEO4J_URI"]                 # e.g. neo4j+s://<id>.databases.neo4j.io
AUTH_USER = os.environ["NEO4J_USERNAME"]      # e.g. neo4j
AUTH_PASSWORD = os.environ["NEO4J_PASSWORD"]  # Aura password

def ping():
    driver = GraphDatabase.driver(URI, auth=basic_auth(AUTH_USER, AUTH_PASSWORD))
    timestamp = datetime.utcnow().isoformat()
    query = """
    MERGE (p:Ping {id: 1})
    SET p.lastPing = $timestamp
    """
    with driver.session() as session:
        session.run(query, timestamp=timestamp)
    driver.close()

if __name__ == "__main__":
    ping()

In [7]:
#!/usr/bin/env python3
"""
Richer Neo4j ping script to gather a snapshot of basic database telemetry and optionally record
it back into Neo4j for auditing. Uses `src/database/ping.py`

By default, it prints the collected metrics as pretty JSON and writes a `PingLog` node 
unless `--dry-run` is supplied.

Usage example:

    python scripts/neo4j_ping.py \\
        --uri "$NEO4J_URI" \\
        --user "$NEO4J_USER" \\
        --password "$NEO4J_PWD" \\
        --monitor-id primary

In the corresponding database, **A PingLog node is created (or merged) per monitor_id** 
and holds the latest stats plus a rolling history:

According to , it carries:
    monitor_id (string): natural key for the environment or credential set.
    created_at (datetime): when the node was first created.
    lastPing-machine-id (string): identifier of the host that ran the most recent ping.
    lastPing-timestamp (string): human-friendly Central Time representation of that ping.
    lastPing-number_nodes (integer): most recent total node count.
    lastPing-number_rels (integer): most recent total relationship count.
    neo4j_version / neo4j_edition (strings): metadata from dbms.components().
    log (list of strings): rolling JSON entries (trimmed to the latest 365) storing the lightweight ping summary (ping_id, both timestamps, counts, machine id).

NOTE: remember that the hyphenated property names require backticks in Cypher, e.g., p.\lastPing-timestamp``

"""

import subprocess, json, os

cmd = [
    "python", "../scripts/neo4j_ping.py",
    "--monitor-id", "primary",
    "--uri", os.environ["NEO4J_URI"],
    "--user", os.environ["NEO4J_USERNAME"],
    "--password", os.environ["NEO4J_PASSWORD"],
    "--monitor-id", "default",    
    "--dry-run",
]
payload = subprocess.check_output(cmd, text=True)
print(payload)


2025-10-21 16:56:52,216 - INFO - Testing Neo4j connection...
2025-10-21 16:56:52,616 - INFO - Neo4j connection test successful
2025-10-21 16:56:52,616 - INFO - Neo4j connection initialized to neo4j+s://5a36bc94.databases.neo4j.io


{
  "ping_id": "61c73351-f523-49ea-a758-2871bc216780",
  "timestamp_iso": "2025-10-21T21:56:52.875941+00:00",
  "timestamp_display": "2025-10-21 04:56:52 PM CDT",
  "machine_id": "Macmini.lan",
  "number_node_types": 6,
  "number_nodes": 43710,
  "number_relationship_types": 5,
  "number_relationships": 111747,
  "neo4j_version": "5.27-aura",
  "neo4j_edition": "enterprise",
  "raw": {
    "version": "5.27-aura",
    "edition": "enterprise",
    "node_labels": [
      "Paper",
      "Author",
      "PubVenue",
      "PubYear",
      "Field",
      "SchemaVersion"
    ],
    "relationship_types": [
      "PUBLISHED_IN",
      "PUB_YEAR",
      "IS_ABOUT",
      "CITES",
      "AUTHORED"
    ],
    "total_nodes": 43710,
    "total_relationships": 111747
  }
}



2025-10-21 16:56:52,875 - INFO - Retrieved database info: 5.27-aura
2025-10-21 16:56:52,876 - INFO - Dry run requested; skipping persistence.
2025-10-21 16:56:52,877 - INFO - Neo4j connection closed
